**Colab Link URL**
- https://colab.research.google.com/drive/1GGc1DPyNOquJ5hBpDDN3K4EK-vPyxyAB?usp=sharing

이 노트북은 [Google Colab](https://colab.research.google.com/)에서 실행하시기에 최적화되어있습니다.
상기 URL을 이용해서 코드를 실행 해보세요.

In [34]:
!pip install langchain langchain-community langchain-openai pypdf fpdf faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.4 MB/s eta 0:00:00


In [35]:
!apt-get install fonts-nanum* -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'fonts-nanum-extra' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-coding' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum-eco' for glob 'fonts-nanum*'
Note, selecting 'fonts-nanum' for glob 'fonts-nanum*'
fonts-nanum is already the newest version (20200506-1).
fonts-nanum-coding is already the newest version (2.5-3).
fonts-nanum-eco is already the newest version (1.000-7).
fonts-nanum-extra is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [36]:
import os
import textwrap

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from google.colab import userdata
from fpdf import FPDF

In [37]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [38]:
pdf = FPDF()
pdf.add_page()
pdf.add_font('NanumGothic', '', '/usr/share/fonts/truetype/nanum/NanumGothic.ttf', uni=True)
pdf.set_font('NanumGothic', '', 12)


pdf_content = """
페이지 1:
제목: 자율 인지 에이전트(ACA)의 아키텍처 연구

초록: 본 논문은 차세대 AI 패러다임인 자율 인지 에이전트(Autonomous Cognitive Agent, ACA)의 핵심 아키텍처를 제안한다. ACA는 LLM을 핵심 두뇌로 사용하며, 동적 계획 수립(Dynamic Planning), 다중 도구 사용(Multi-Tool Use), 그리고 장기 기억(Long-term Memory)이라는 세 가지 필수 구성 요소로 이루어진다.

---
페이지 2:
ACA의 핵심 구성 요소 1: 동적 계획 수립

전통적인 에이전트는 고정된 계획을 따르지만, ACA의 계획 수립 모듈은 환경의 변화나 예기치 않은 결과에 따라 실시간으로 목표와 전략을 수정한다. 이는 '리플렉션(Reflection)' 메커니즘을 통해 실패 경험으로부터 학습하고, 다음 행동 계획을 최적화함으로써 달성된다.

---
페이지 3:
ACA의 핵심 구성 요소 2: 다중 도구 사용

ACA는 단일 도구에 의존하지 않는다. 웹 검색, 코드 실행, 데이터베이스 조회 등 다양한 API를 도구로 등록하고, 현재 해결해야 할 하위 과업에 가장 적합한 도구를 스스로 선택하여 호출한다. 도구 선택 과정은 LLM의 추론 능력에 기반하며, 이를 통해 복잡하고 다단계적인 문제 해결이 가능해진다.

---
페이지 4:
ACA의 핵심 구성 요소 3: 장기 기억

인간과 같이 ACA는 과거의 상호작용과 성공/실패 경험을 장기 기억에 저장한다. 이 기억은 벡터 데이터베이스에 저장되어, 새로운 과업이 주어졌을 때 관련성 높은 과거 경험을 즉시 참조할 수 있게 한다. 이는 에이전트가 동일한 실수를 반복하지 않고 시간이 지남에 따라 점차 현명해지도록 만드는 핵심 요소이다.
"""
pdf.multi_cell(0, 10, pdf_content)
pdf_file_path = "llm_agent_research_paper.pdf"
pdf.output(pdf_file_path)

/usr/local/lib/python3.11/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43961
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.11/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43964
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.11/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -43984
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.11/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -44070
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.11/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -44085
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.11/dist-packages/fpdf/ttfonts.py:670: UserWarning: cmap value too big/small: -44096
  warnings.warn("cmap value too big/small: %s" % cm)
/usr/local/lib/python3.11/dist-packages/

''

In [49]:
print(f"--- [단계 1] PDF 데이터 로딩 ---")
loader = PyPDFLoader(pdf_file_path)
docs = loader.load()
print(f"'{pdf_file_path}'에서 {len(docs)}개의 페이지를 로드했습니다.\n")

--- [단계 1] PDF 데이터 로딩 ---
'llm_agent_research_paper.pdf'에서 2개의 페이지를 로드했습니다.



In [50]:
print(f"--- [단계 2] 문서 분할 ---")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
print(f"문서를 {len(splits)}개의 청크로 분할했습니다.\n")

--- [단계 2] 문서 분할 ---
문서를 3개의 청크로 분할했습니다.



In [51]:
# 벡터 저장소 및 검색기 구축
print(f"--- [단계 3] 벡터 저장소 구축 및 검색기 생성 ---")
vectorstore = FAISS.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY),
)
retriever = vectorstore.as_retriever()
print("검색기 준비 완료.\n")

--- [단계 3] 벡터 저장소 구축 및 검색기 생성 ---
검색기 준비 완료.



In [52]:
# 고급 RAG 체인 설계
print(f"--- [단계 4] RAG 체인 설계 ---")

--- [단계 4] RAG 체인 설계 ---


In [53]:
# 검색된 문서 리스트를 하나의 문자열로 합치는 헬퍼 함수
def format_docs(docs):
    return "\n\n".join(f"출처: {doc.metadata.get('source', 'N/A')}, 페이지: {doc.metadata.get('page', 'N/A')+1}\n내용: {doc.page_content}" for doc in docs)

In [54]:
# RAG 프롬프트 템플릿 정의
rag_prompt = ChatPromptTemplate.from_template(
    """당신은 주어진 문맥(Context)을 바탕으로 질문에 대해 명확하고 간결하게 답변하는 AI 연구원입니다.
    답변은 반드시 주어진 문맥에 근거해야 합니다.

    [문맥]:
    {context}

    [질문]:
    {question}

    [답변]:
    """
)

In [55]:
# RAG 프롬프트 템플릿 정의
rag_prompt = ChatPromptTemplate.from_template(
    """당신은 주어진 문맥(Context)을 바탕으로 질문에 대해 명확하고 간결하게 답변하는 AI 연구원입니다.
    답변은 반드시 주어진 문맥에 근거해야 합니다.

    [문맥]:
    {context}

    [질문]:
    {question}

    [답변]:
    """
)

# LLM 정의
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o",
    temperature=0,
)

# 최종 답변을 생성하는 체인
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | rag_prompt

    | llm
    | StrOutputParser()
)

In [56]:
# 검색된 원본 문서(context)와 생성된 답변(answer)을 함께 반환하는 최종 체인
# RunnableParallel을 사용하여 병렬적으로 두 개의 결과를 구성한다.
rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

print("체인 설계 완료.\n")

체인 설계 완료.



In [57]:
# 체인 실행 및 결과 해석
print(f"--- [단계 5] 체인 실행 ---")
question = "자율 인지 에이전트(ACA)의 핵심 구성 요소 세 가지는 무엇인가요?"
response = rag_chain_with_source.invoke(question)

--- [단계 5] 체인 실행 ---


In [58]:
# 결과 출력
print(f"질문: {question}\n")
print("="*50)
print(f"생성된 답변:\n")
# textwrap을 사용하여 긴 답변을 자동으로 줄 바꿈
wrapper = textwrap.TextWrapper(width=80, replace_whitespace=False)
print(wrapper.fill(text=response["answer"]))
print("\n" + "="*50)

print("답변의 근거가 된 출처:")
for doc in response["context"]:
    print(f"- 출처: {doc.metadata.get('source', 'N/A')}, 페이지: {doc.metadata.get('page', 'N/A')+1}")

질문: 자율 인지 에이전트(ACA)의 핵심 구성 요소 세 가지는 무엇인가요?

생성된 답변:

자율 인지 에이전트(ACA)의 핵심 구성 요소 세 가지는 동적 계획 수립, 다중 도구 사용, 장기 기억입니다.

답변의 근거가 된 출처:
- 출처: llm_agent_research_paper.pdf, 페이지: 1
- 출처: llm_agent_research_paper.pdf, 페이지: 2
- 출처: llm_agent_research_paper.pdf, 페이지: 1
